In [1]:
from numba import cuda
from numba.cuda import as_cuda_array as ca
import numpy as np
import math
import torch

def cdiv(a,b):
    "Int ceiling division of `a` over `b`"
    return (a+b-1)//b

import os
os.environ["CUDA_HOME"] = "/usr/local/cuda-12.6"


In [2]:
@cuda.jit
def matmul_k_numba(m, n, out, tw):
    cbi,cbd,tid = cuda.blockIdx,cuda.blockDim,cuda.threadIdx
    tc,tr = tid.x,tid.y
    r,c = cbi.y * cbd.y + tr, cbi.x * cbd.x + tc
    h,k  = m.shape
    k2,w = n.shape

    shar = cuda.shared.array(0, dtype=np.float32)
    ms,ns = shar[:tw*tw],shar[tw*tw:2*tw*tw]

    p = np.float32(0.0)
    for ph in range(math.ceil(k/tw)):
        idx = ph*tw
        ms[tr*tw+tc] = m[r, tc+idx] if r<h and idx+tc<k else 0.
        ns[tr*tw+tc] = n[tr+idx, c] if c<w and idx+tr<k else 0.
        cuda.syncthreads()
        for i in range(tw): p += ms[tr*tw+i] * ns[i*tw+tc]
        cuda.syncthreads()
    if r < h and c < w: out[r, c] = p

In [3]:
def matmul_2d_numba(m, n, tw=16):
    h,k  = m.shape
    k2,w = n.shape
    assert k==k2, "Size mismatch!"
    out = torch.zeros(h, w, dtype=m.dtype, device=m.device)
    dyn_shared_mem_size = 2 * tw * tw * 4
    tpb = tw,tw
    blocks = cdiv(w,tpb[0]), cdiv(h,tpb[1])
    matmul_k_numba[blocks, tpb, 0, dyn_shared_mem_size](ca(m), ca(n), ca(out), tw) 
    return out

In [5]:

N, L, M = 12, 34, 65

Q = torch.rand(N, L).contiguous().cuda()
K = torch.rand(L, M).contiguous().cuda()

torch.isclose(matmul_2d_numba(Q, K), Q@K).all()

TypeError: *obj* doesn't implement the cuda array interface.

In [ ]:
N, L, M = 12, 34, 65

Q = torch.rand(N, L).contiguous().cuda()

In [ ]:
%%timeit -n 10
matmul_2d_numba(m1c,m2c)
torch.cuda.synchronize()